<font size=6 color ="#03f4fc" >Genetic Algorithm</font>
- <font size=4>Name : Hesam Ramezanian</font>
- <font size=4>UID  : 810100248</font>

<font size ="5" color ="#03c2fc">Part 1: Basic Concepts</font>
- each coefficient is considered a gene.
- A chromosome consists of n+1 genes where n is degree of the given equation.

In [104]:
import random
from timeit import default_timer as timer
import copy

In [140]:
POPULATION_SIZE = 500
CROSSOVER_PROBABILITY = 0.7
MUTATION_PROBABILITY = 0.1
CARRY_PERCENTAGE = 0.2
MAX_NUMBER_OF_GENERATION = 2000

<font size ="5" color ="#03c2fc">Part 2: Initialize first population</font>
- i used random integers to make chromosomes of first population with size : `POPULATION_SIZE` 

In [106]:
def MakeFirstPopulation(min_coefficient:int, max_coefficient:int,degree:int)->list:
    numOfCoefficients = degree + 1
    firstPopulation = []
    for _ in range(POPULATION_SIZE):
        chromosome = [random.randint(min_coefficient,max_coefficient) for _ in range(numOfCoefficients)]
        firstPopulation.append(chromosome)
    return firstPopulation

<font size ="5" color ="#03c2fc">Part 3: Fitness Function</font>
- i stored the sum of the differences between the solution of the equation derived from a chromosome and the input points in the `sumOfDifference` variable. Then, if its value was 0 , i returnd 2 (maximum fitness value) as the fitness value ,otherwise i returnde 1/`sumOfDifference`

In [107]:
def CalaulateEquation(x:int,coefficients_list:list)->int:
    y = 0
    for degree in range(len(coefficients_list)):
        y += (x**degree)*coefficients_list[degree]
    return y



def FitnessFunction(chromosome:list,point_list : list)-> float:
    sumOfDifference = 0
    y = 0
    x = 0
    for point in point_list:
        x = point[0]
        y = point[1]
        sumOfDifference += abs(y-CalaulateEquation(x,chromosome))
    if sumOfDifference == 0 :
        return 2
    else:
        return 1/sumOfDifference
            

<font size ="5" color ="#03c2fc">Part 4: Implementing Crossover and Mutation and Generating Next Population</font>

<font size ="4" > Crossover : </font>
- i tried two types of crossover function. At first for each  chromosome in the population , I created a random number between 0 and 1. if it was less than `CROSSOVER_PROBABILITY`,I performed the crossover operation as one point or uniform

In [108]:
def CrossoverOnePoint(chromosome1:list,chromosome2:list):
    crossoverPoint = random.randint(1,len(chromosome1))
    chromosome1[:crossoverPoint],chromosome2[:crossoverPoint] = chromosome2[:crossoverPoint],chromosome1[:crossoverPoint]

def CrossoverUniform(chromosome1:list,chromosome2:list):
    for i in range(len(chromosome1)):
        if random.random() < 0.5 :
            chromosome1[i],chromosome2[i] = chromosome2[i],chromosome1[i]
    
def CrossoverOnList(population:list):
    populationLen = len(population)
    for i in range(0,populationLen,2):
        #for odd population size
        if i + 1 == populationLen:
            break
        if(random.random() < CROSSOVER_PROBABILITY):
            ch1 = copy.deepcopy(population[i])
            ch2 = copy.deepcopy(population[i+1])
            #CrossoverOnePoint(ch1,ch2)
            CrossoverUniform(ch1, ch2)
            population[i] = ch1
            population[i+1] = ch2
    


<font size ="4" > Mutation : </font>
- For each chromosome in the population, if a random number between 0 and 1 was less than `MUTATION_PROBABILITY` i performed mutation,in the mutation, i randomly selected a gene from the chromosome and replaced it with a random number.
- `CARRY_PERCENTAGE` of best chromosome in the population without mutation carry to next generation

In [109]:
def Mutate(chromosome:list,min_coefficient:int,max_coefficient:int):
    geneIndex = random.randint(0,len(chromosome)-1)
    chromosome[geneIndex] = random.randint(min_coefficient,max_coefficient)
    
def MutateOnList(population:list,min_coefficient:int,max_coefficient:int):
    populationLen = len(population)
    for i in range(int(populationLen*CARRY_PERCENTAGE),populationLen):
        if(random.random() < MUTATION_PROBABILITY):
            ch = copy.deepcopy(population[i])
            Mutate(population[i],min_coefficient,max_coefficient)
            population[i] = ch

<font size ="4" > Generating Next Population : </font>
- I experimented two methods rank selection and wheel selection for creating the mating pool. Each method has its own pros and cons. Rank selection usually makes the algorithm converge slower to the solution than wheel selection. However, for equations with high degrees and wide ranges of coefficients, rank selection tends to have a higher chance of finding the optimal solution than wheel selection.

In [110]:
def CreateMatingPoolWheel(population:list,points_list:list)->list:
    listLen = len(population)
    fitnesList = [FitnessFunction(population[i],points_list) for i in range(0,listLen)]
    sumOfFitness = sum(fitnesList)
    probList = [(fitnesList[i]/sumOfFitness) for i in range(0,listLen)]
    matingPool = random.choices(population,weights=probList,k=POPULATION_SIZE)
    random.shuffle(matingPool)
    return matingPool

def RankList(population:list,point_list:list):
    population.sort(key = lambda ch : FitnessFunction(ch,point_list) ,reverse = True)

def CreateMatingPoolRank(population:list)->list:
    listLen = len(population)
    sumOfRanks = (listLen * (listLen + 1))/2
    probList = [(listLen-i+1)/sumOfRanks for i in range(1,listLen+1)]
    matingPool = random.choices(population,weights=probList,k=POPULATION_SIZE)
    random.shuffle(matingPool)
    return matingPool

<font size ="5" color ="#03c2fc">Part 5: Running the Genetic Algorithm</font>
- I implemented two functions to execute the algorithm that differ only in the way they select the next generation.

    FindSolution1 = wheel selection

    FindSolution2 = rank selection
- in each generation i ranked the population list and saved best answer in `newBestAnswer` and if it was better than the last best answer i change `bestAnswer`
- for each 50 generation i print fitness of best answer i found to see progress
- if i found the optimal solution i stop the algorithm
- The mating pool is created, and crossover and mutation is applied

In [111]:
def FindSolution1(min_coefficient:int,max_coefficient:int,points_list:list,degree:int)->(list,float):
    population = MakeFirstPopulation(min_coefficient,max_coefficient,degree)
    bestAnswer = []
    bestAnswerFitness = 0
    for i in range(MAX_NUMBER_OF_GENERATION):
        RankList(population,points_list)
        newBestAnswer = population[0]
        newBestAnswerFitness = FitnessFunction(newBestAnswer,points_list)
        if bestAnswerFitness < newBestAnswerFitness:   
            bestAnswerFitness = newBestAnswerFitness
            bestAnswer = newBestAnswer
        if(not i%50):
            print(i," : ",bestAnswerFitness)
        if(bestAnswerFitness == 2):
            break
        population = CreateMatingPoolWheel(population,points_list)
        CrossoverOnList(population)
        MutateOnList(population,min_coefficient,max_coefficient)
    return bestAnswer,bestAnswerFitness


def FindSolution2(min_coefficient:int,max_coefficient:int,points_list:list,degree:int)->(list,float):
    population = MakeFirstPopulation(min_coefficient,max_coefficient,degree)
    bestAnswer = []
    bestAnswerFitness = 0
    for i in range(MAX_NUMBER_OF_GENERATION):
        RankList(population,points_list)
        newBestAnswer = population[0]
        newBestAnswerFitness = FitnessFunction(newBestAnswer,points_list)
        if bestAnswerFitness < newBestAnswerFitness:   
            bestAnswerFitness = newBestAnswerFitness
            bestAnswer = newBestAnswer
        if(not i%50):
            print(i," : ",bestAnswerFitness)
        if(bestAnswerFitness == 2):
            break
        population = CreateMatingPoolRank(population)
        CrossoverOnList(population)
        MutateOnList(population,min_coefficient,max_coefficient)
    return bestAnswer,bestAnswerFitness
    
    


In [141]:
#minCoefficient,maxCoefficient = -5,3
#pointsList = [[0,1],[1,0],[2,-5],[-1,-8]]
#degree = 3

minCoefficient,maxCoefficient = -12,+12
pointsList = []
degree = 10

#fill pointList:
numberOfPoints = 60
minX = -30
maxX = 30
ans = []
xlist = []
x = 0
for _ in range(numberOfPoints):
    while True: 
        x = random.randint(minX,maxX)
        if x not in xlist:
            xlist.append(x)
            break
    ans.append([x,10*(x**10)+2*(x**8)-5*(x**7)-8*(x**5)+4*(x**3)+11])
pointsList = ans

In [113]:
sol1,fitness1 = FindSolution1(minCoefficient,maxCoefficient,pointsList,degree)
print("best answer :",sol1,"fitness :", fitness1)

0  :  1.0788281815377162e-13
50  :  2.0596616845029594e-09
100  :  2.0596616845029594e-09
150  :  2.0596616845029594e-09
200  :  1.4574922600966647e-08
250  :  1.0522763526039157e-07
300  :  2.5157169415925998e-06
350  :  5.553087516659262e-05
400  :  0.0011904761904761906
450  :  0.016666666666666666
best answer : [11, 0, 0, 4, 0, -8, 0, -5, 2, 0, 10] fitness : 2


In [114]:
sol2,fitness2 = FindSolution2(minCoefficient,maxCoefficient,pointsList,degree)
print("best answer :",sol2,"fitness :", fitness2)

0  :  1.0911578538320062e-13
50  :  1.5929351166283168e-10
100  :  4.128477254348606e-09
150  :  1.0644793461201431e-07
200  :  1.7529814269864828e-07
250  :  2.343468051500054e-06
300  :  2.554532890888237e-06
350  :  2.554532890888237e-06
400  :  0.0011098779134295228
450  :  0.016666666666666666
500  :  0.016666666666666666
550  :  0.016666666666666666
best answer : [11, 0, 0, 4, 0, -8, 0, -5, 2, 0, 10] fitness : 2


<font size="4" color="red">Note that the list of coefficients from left to right represents the coefficient of each degree of the equation for example:</font>

[3,5] => 5x+3

<font size ="5" color ="#03c2fc">Questions</font>

<font size="5"> How can extremely large or very small population sizes affect the performance of the algorithm?</font>

- Both very small and very large population sizes can negatively impact the algorithm's performance. A small population may lack the diversity needed to find the optimal solution. A large population requires more computational resources and time without necessarily improving the end result.

<font size="5">If the population size increases each generation, what effect does it have on the algorithm's accuracy and speed?</font>
- Increasing the population size each generation will slow down the algorithm's execution as it has to evaluate more chromosomes in each generation. It may slightly improve accuracy by maintaining diversity,Keeping population size constant is preferred for efficiency. 

<font size="5">Explain and compare the effects of crossover and mutation operations. Can you only use one of them? Why?</font>
- Crossover combines genetic material between chromosomes to produce new better solutions. Mutation randomly alters a chromosome to introduce variation. Using crossover alone cannot generate enough diversity. Mutation helps avoid local optima by changing chromosomes, but cannot efficiently combine good solutions. Both are necessary for an effective search.

<font size="5">In your opinion, what solutions exist to reach the answer faster for this specific problem?</font>
- Carefully tuning the algorithm parameters like `POPULATION_SIZE`, `MUTATION_PROBABILITY`,... and functions like crossover,fitness and ..... can improve performance for this specific problem.


<font size="5">Even using these methods, it's possible chromosomes won't change after a few more generations. Explain why this happens and the problems it causes. What do you suggest to fix it?</font>
- The algorithm may converge prematurely to a local rather than global optimum. This lack of diversity causes stagnation. Restarting with a new population or increasing mutation rate helps escape suboptimal convergence.

<font size="5">What solution do you propose for terminating the program if the problem has no solution?</font>
- We can check the best answer every several generations, for example each 500 generation and stop the algorithm if the bestAnswer is the same.

<font size="5">As the polynomial degree increases, how does the time to find the coefficients change?</font>
-  Increasing polynomial degree exponentially increases the search space size, making it much harder to find optimal coefficients. The algorithm will require more generations and take longer to converge. y

<font size="5">In your opinion, how does increasing or decreasing the number of points affect the algorithm's execution?</font>
-  More data points constrain the search space, making it easier to find coefficients fitting the data. With fewer points, the solution space is less defined, requiring more exploration to fit the polynomial.also With fewer points algorithm can find more than one optimal solution

In [137]:
def FindSolutionTerminate(min_coefficient:int,max_coefficient:int,points_list:list,degree:int)->(list,float):
    population = MakeFirstPopulation(min_coefficient,max_coefficient,degree)
    bestAnswer = []
    bestAnswerFitness = 0
    terminateGenerationNumber = 100
    lastTerminateFitness = -1
    for i in range(MAX_NUMBER_OF_GENERATION):
        RankList(population,points_list)
        newBestAnswer = population[0]
        newBestAnswerFitness = FitnessFunction(newBestAnswer,points_list)
        
        if(not (i % terminateGenerationNumber)):
            if(lastTerminateFitness == bestAnswerFitness):
                break
            lastTerminateFitness = newBestAnswerFitness
            
        if bestAnswerFitness < newBestAnswerFitness:   
            bestAnswerFitness = newBestAnswerFitness
            bestAnswer = newBestAnswer
            
        if(not i%50):
            print(i," : ",bestAnswerFitness)
            
        if(bestAnswerFitness == 2):
            break
            
        population = CreateMatingPoolRank(population)
        CrossoverOnList(population)
        MutateOnList(population,min_coefficient,max_coefficient)
    return bestAnswer,bestAnswerFitness

<font size=4>result of algorithm with terminating if algorithm stuck and multi start :</font>
- if algorithm stuck in one answer for `terminateGenerationNumber` generation algorithm terminating
- algorithm start for `numberOfMultiStart` times to find optimal solution

In [139]:
numberOfMultiStart = 5
solMulti = []
FitnessMulti = 0
for _ in range(numberOfMultiStart):
    solMulti,FitnessMulti = FindSolutionTerminate(minCoefficient,maxCoefficient,pointsList,degree)
    if(FitnessMulti == 2):
        break
    print("new start:\n")
print("best answer :",sol1,"fitness :", fitness1)

0  :  7.928973387897707e-15
50  :  8.4702391943638e-15
100  :  1.1170460069989566e-13
150  :  2.2929653769079404e-13
200  :  1.4836719726938358e-12
250  :  3.0225313697279943e-12
300  :  1.3005462585725415e-11
350  :  4.054027357736062e-09
400  :  1.4473138535116336e-08
450  :  5.488895963465909e-06
500  :  1.852778240972338e-05
550  :  5.580668564093978e-05
600  :  0.016666666666666666
650  :  0.016666666666666666
new start:

0  :  2.1502261240755322e-13
50  :  6.210261602925821e-12
100  :  1.67954026131192e-10
150  :  5.234441473317304e-08
200  :  5.043016934450866e-06
250  :  0.0009233610341643582
300  :  0.0009233610341643582
350  :  0.016666666666666666
best answer : [11, 0, 0, 4, 0, -8, 0, -5, 2, 0, 10] fitness : 2
